#### Como primera misión el cliente ha solicitado corroborar que dominamos el manejo de transformaciones con RDD's. Presta mucha atención a las instrucciones listadas y resuelve las actividades.

In [1]:
# NO MODIFICAR CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession
from pyspark import RDD


#### Actividad 1:
##### TO DO: Crear la sesion de spark y el contexto de spark. Almacenarlos en las variables "spark" y "sc" respectivamente. 
##### Colocando "local[*]" en la opción master y "Ejercicio_3" en el appName

In [2]:
## TU CODIGO VA EN ESTA CELDA:
spark = SparkSession.builder.appName("Ejercicio_3").master("local[*]").getOrCreate() # levantar sesión de spark
sc = spark.sparkContext # crear contexto de spark

In [3]:
# NO MODIFICAR CONTENIDO DE ESTA CELDA

from pyspark.context import SparkContext

assert type(spark) == SparkSession
assert spark.sparkContext.getConf().get("spark.master") == "local[*]"
assert spark.sparkContext.getConf().get("spark.app.name") == "Ejercicio_3"
assert type(sc) == SparkContext

#### Actividad 2:
##### TO DO: Leer el archivo de texto: "../../resources/data/txt/green_eggs_and_ham.txt" utilizando el sparkContext para cargar un RDD[str]

In [4]:
## TU CODIGO VA EN ESTA CELDA:

#data: RDD[str] = sc.emptyRDD() #... reemplazar por lectura de archivo con sc
data: RDD[str] = sc.textFile(name = "../../resources/data/txt/green_eggs_and_ham.txt")

In [5]:
# NO MODIFICAR CONTENIDO DE ESTA CELDA
assert type(data) == RDD
assert data.count() == 103

#### Actividad 3:
##### TO DO: Crear un RDD que contenga el conteo de cada palabra contenida en el documento leido anteriormente.
##### El resultado tendrá que ser un RDD de tuplas con la siguiente estructura (word, count), donde la primer posición deberá ser la palabra y la segunda posicion el total de veces que aparece esa palabra en el RDD leido 
##### NOTA: Limpiar el contenido de data ya que existen espacios dobles y convertir todo el contenido en MAYUSCULAS
Ejemplo:

Archivo de entrada:

HOLA  MUNDO HOLA  
feliZ Mundo  
FELIZ MUNDO ADIOS

Salida:

(MUNDO, 3)  
(HOLA, 2)  
(FELIZ, 2)  
(ADIOS, 1)



In [6]:
## TU CODIGO VA EN ESTA CELDA:
# Primero es necesario limpiar el texto, remover los espacios y los dobles espacios.
import re

# De la expresión regular " + " para buscar más coincidencias y " " para eliminar los espacios en blanco extra
# flatMap() sirve pata descomponer el arreglo que genera el split en muchos registros independientes
# PENDIENTE, línea 86 aparece un sAM, se necesita convertir en uppercase
texto_limpio = data\
    .map(lambda remover_espacios: re.sub(" + ", " ", remover_espacios))\
    .map(lambda mayus: mayus.upper())\
    .flatMap(lambda separar_elementos: separar_elementos.split(" "))

    #.map(lambda mayus: re.sub("sAM", "SAM", mayus))\
            
# Validación
#texto_limpio.foreach(lambda item: print(item))

# Trabajamos ya con el texto limpio, separar cada elemento y configurarlos en un array (palabra, 1), luego hacer agrupamiento por llaves,
# posteriormente realizar un mapeo donde se haga la sumatoria de los valores [1] post-agrupación para saber el número de repetición e imprimirlos
result = texto_limpio\
    .map(lambda array: (array,1))\
    .groupByKey()\
    .map(lambda values: (values[0], sum(list(values[1]))))
    
result.foreach(lambda item: print(item))

# Resultado final
#result: RDD = data # reemplazar por transformaciones a la variable data

In [7]:
# NO MODIFICAR CONTENIDO DE ESTA CELDA
assert result.count() == 52

ordered_result = result.sortBy(lambda item: item[1], ascending=False, numPartitions=1).take(5)
expected = [('I', 83), ('NOT', 82), ('THEM', 58), ('A', 56), ('LIKE', 44)]
assert ordered_result == expected

sam_values = result.filter(lambda item: item[0] == 'SAM').take(1)
assert sam_values[0][1] == 19
result.foreach(print)